## This notebook investigates scraping and adding in a score differential (margin) for each record

Source: http://www.pro-football-reference.com/boxscores/201411020nwe.htm#pbp

In [128]:
import cPickle as pickle
import pandas as pd
import numpy as np
import random 
import copy
import matplotlib
%matplotlib inline

# for scraping
from pymongo import MongoClient
from bson.objectid import ObjectId
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common.exceptions import TimeoutException

In [154]:
def clean(filename):
    pbp = pd.read_csv(filename)

    # convert column names to upper case (since they are inconsistent in the data by season)
    pbp.columns = [x.upper() for x in pbp.columns]
    
    # drop all columns with no info, we won't use, or aren't in all the data
    pbp.drop(
        ['UNNAMED: 10', \
         'UNNAMED: 12', \
         'UNNAMED: 16',\
         'UNNAMED: 17', \
         'CHALLENGER', \
         'ISMEASUREMENT', \
         'NEXTSCORE', \
         'TEAMWIN',\
         'ISINCOMPLETE', \
         'ISTOUCHDOWN', \
         'ISSACK', \
         'ISCHALLENGE', \
         'ISCHALLENGEREVERSED', \
         'ISINTERCEPTION', \
         'ISPENALTY', \
         'ISTWOPOINTCONVERSION', \
         'SERIESFIRSTDOWN', \
         'ISTWOPOINTCONVERSIONSUCCESSFUL', \
         'ISPENALTYACCEPTED', \
         'PENALTYTEAM', \
         'ISFUMBLE', \
         'PENALTYTYPE', \
         'PENALTYYARDS', \
         #'SEASONYEAR', \
         #'GAMEID', \
         #'GAMEDATE', \
         'ISNOPLAY', \
         'DEFENSESCORE',
         'ISMESUREMENT',
         'ISPRESEASON',
         'OFFENSESCORE',
         'PENALIZEDPLAYER',
         'PLAYID',
         'SCORECHANGE',
         'SCOREDIFF'], \
          axis=1, inplace=True,  errors='ignore')
    
    # get rid of all the kicks except punts and field goals
    pbp = pbp[(pbp.PLAYTYPE != 'KICK OFF') & (pbp.PLAYTYPE != 'EXTRA POINT') & \
              (pbp.PLAYTYPE != 'TWO-POINT CONVERSION')]

    # get rid of all the timeout plays
    pbp = pbp[pbp.OFFENSETEAM.notnull()]

    # get rid of all of the no-plays
    pbp = pbp[(pbp.PLAYTYPE != 'NO PLAY') & (pbp.PLAYTYPE != 'EXCEPTION') & \
              (pbp.PLAYTYPE != 'CLOCK STOP')& (pbp.PLAYTYPE != 'PENALTY')]

    # get rid of all the malformed pass types
    passtypes = ['DEEP MIDDLE', 'SHORT LEFT', 'SHORT RIGHT', 'SHORT MIDDLE', \
                 'DEEP LEFT', 'DEEP RIGHT']
    pbp = pbp[(pbp['PASSTYPE'].isin(passtypes)) | (pbp['PASSTYPE'].isnull())]

    # replace the nan PlayTypes with 'DIRECT SNAP'
    pbp.PLAYTYPE = pbp.PLAYTYPE.fillna('DIRECT SNAP')

    # drop the existing IsRush/IsPass and create new ones
    pbp.drop(['ISRUSH', 'ISPASS'], axis=1, inplace=True)

    play_to_rush = {
        'RUSH': 1,
        'PASS' : 0,
        'PUNT' : 0,
        'QB KNEEL' : 1,
        'SCRAMBLE' : 0,
        'FIELD GOAL' : 0,
        'SACK' : 0,
        'FUMBLES' : 1,
        'DIRECT SNAP' : 1
    }

    pbp['ISRUSH'] = pbp['PLAYTYPE'].map(play_to_rush)

    play_to_pass = {
        'RUSH': 0,
        'PASS' : 1,
        'PUNT' : 0,
        'QB KNEEL' : 0,
        'SCRAMBLE' : 1,
        'FIELD GOAL' : 0,
        'SACK' : 1,
        'FUMBLES' : 0,
        'DIRECT SNAP' : 0
    }

    pbp['ISPASS'] = pbp['PLAYTYPE'].map(play_to_pass)

    play_to_kick = {
        'RUSH': 0,
        'PASS' : 0,
        'PUNT' : 1,
        'QB KNEEL' : 0,
        'SCRAMBLE' : 0,
        'FIELD GOAL' : 1,
        'SACK' : 0,
        'FUMBLES' : 0,
        'DIRECT SNAP' : 0
    }

    pbp['ISKICK'] = pbp['PLAYTYPE'].map(play_to_kick)


    # Combine the dummy classes into one var and drop the dummies
    def play_type(x):
        if x.ISRUSH == 1:
            return 'RUSH'
        if x.ISPASS == 1:
            return 'PASS'
        if x.ISKICK == 1:
            return 'KICK'
        else:
            return 'NaN'

    pbp['PLAY'] = pbp.apply(lambda x: play_type(x), axis=1)

    pbp.drop(['ISRUSH', 'ISPASS', 'ISKICK'], axis=1, inplace=True)

    # Convert some columns to categorical
    pbp.FORMATION = pbp.FORMATION.astype("category")
    pbp.OFFENSETEAM = pbp.OFFENSETEAM.astype("category")
    pbp.DEFENSETEAM = pbp.DEFENSETEAM.astype("category")
    pbp.PLAY = pbp.PLAY.astype("category")

    return pbp

In [3]:
def prep(df):

    # A few columns have to go for us to build a model (though we use them for user presentation)
    df.drop(['YardLineFixed', 'YardLineDirection','Description', 'PlayType', \
             'PassType', 'RushDirection', 'Yards', 'DefenseTeam'], axis=1, inplace=True)

    # create dummy variables for formations
    form_dummies = pd.get_dummies(df.Formation)
    form_dummies.columns = map(lambda x: 'FORMATION_' + x.replace (' ', '_'), form_dummies.columns)

    # create dummy variables for teams
    team_dummies = pd.get_dummies(df.OffenseTeam)
    team_dummies.columns = map(lambda x: 'TEAM_' + str(x), team_dummies.columns)

    # combine the dummy variables and drop the categorical versions
    df_prepped = pd.concat(
        [df.ix[:,['Quarter', 'Minute', 'Second', 'Down', 'ToGo', 'YardLine', 'Play']],
        team_dummies,
        form_dummies], axis=1)

    return df_prepped

In [155]:
# Go back to the raw data
pbp16 = clean('../data/pbp-2016.csv')
pbp15 = clean('../data/pbp-2015.csv')
pbp14 = clean('../data/pbp-2014.csv')
pbp13 = clean('../data/pbp-2013-fixed.csv') # the 2013 data has errant quotes (fixed with clean.py#remove_inner_quotes())

In [156]:
# combine them all into one data frame
pbp = pd.concat([pbp16, pbp15, pbp14, pbp13])
print pbp.shape

(134998, 20)


In [6]:
# pick a random play out of the data
rand_play = pbp.iloc[random.randint(0,pbp.shape[0])]
rand_play

GAMEID                                                      2015120605
GAMEDATE                                                    2015-12-06
QUARTER                                                              3
MINUTE                                                              13
SECOND                                                               6
OFFENSETEAM                                                        CHI
DEFENSETEAM                                                         SF
DOWN                                                                 1
TOGO                                                                14
YARDLINE                                                            27
DESCRIPTION          (13:06) (SHOTGUN) 6-J.CUTLER SCRAMBLES LEFT EN...
YARDS                                                                9
FORMATION                                                      SHOTGUN
PLAYTYPE                                                      SCRAMBLE
PASSTY

### Go out and scrape the new site

1) pull together a list of game links (for seasons 2013-2016: for weeks 1-17: http://www.pro-football-reference.com/years/<season>/week_<week>.htm --> td class=gamelink)

2) go to each URL pull the full play-by-play table (div id=all_pbp, table_outer_container mobile_table)

3) Descriptions don't match, but clock/down/yards seem to match.  Pull the pbp_score_aw and pbp_score_hm fields and attach them  

In [21]:
# connect to the hosted MongoDB instance
client = MongoClient('mongodb://localhost:27017/')

# open the nfl db
db = client['nfl']
week_pages = db['week_pages']
boxscore_pages_partial = db['boxscore_pages_partial']
boxscore_pages = db['boxscore_pages']

In [58]:
seasons = ['2013', '2014', '2015', '2016']

# for each season and each week
for season in seasons:
    for week in xrange(1,18):
        
        # construct the url for the game's box score and request it
        url = 'http://www.pro-football-reference.com/years/'+season+'/week_'+str(week)+'.htm'
        r = requests.get(url)
        
        # store the html in MongoDB
        week_pages.insert_one({'season':season, 'week':week, 'url': url, 'html':r.content})

In [56]:
url_prefix = 'http://www.pro-football-reference.com'
boxscore_urls = {}

# extract the game boxscore links
for page in week_pages.find():
    html = page['html']
    
    # have BeautifulSoup crack it open
    soup = BeautifulSoup(html, 'html.parser')

    # find the game boxscore links
    for game_link in soup.select('td.gamelink'):
        for link in game_link.find_all('a'):
            boxscore_urls[url_prefix+link.get('href')] = {'week': page['week'], 'season':page['season']}

In [22]:
# scrape the boxscore pages
for i,url in enumerate(boxscore_urls.keys()):
    
    # show the status
    print i+1, 'of', len(boxscore_urls), url
    
    # get the meta data
    week = boxscore_urls[url]['week']
    season = boxscore_urls[url]['season']
    
    # extract the gameid
    gameid = url.rsplit('/', 1)[-1][:-4]
    
    # request the page
    r = requests.get(url)
    
    # store the html in MongoDB
    boxscore_pages_partial.insert_one({'season':season, 'week':week, 'gameid':gameid, 'url': url, 'html':r.content})

1 of 1024 http://www.pro-football-reference.com/boxscores/201309050den.htm
2 of 1024 http://www.pro-football-reference.com/boxscores/201309080chi.htm
3 of 1024 http://www.pro-football-reference.com/boxscores/201309080det.htm
4 of 1024 http://www.pro-football-reference.com/boxscores/201309080buf.htm
5 of 1024 http://www.pro-football-reference.com/boxscores/201309080cle.htm
6 of 1024 http://www.pro-football-reference.com/boxscores/201309080clt.htm
7 of 1024 http://www.pro-football-reference.com/boxscores/201309080pit.htm
8 of 1024 http://www.pro-football-reference.com/boxscores/201309080nyj.htm
9 of 1024 http://www.pro-football-reference.com/boxscores/201309080nor.htm
10 of 1024 http://www.pro-football-reference.com/boxscores/201309080jax.htm
11 of 1024 http://www.pro-football-reference.com/boxscores/201309080car.htm
12 of 1024 http://www.pro-football-reference.com/boxscores/201309080ram.htm
13 of 1024 http://www.pro-football-reference.com/boxscores/201309080sfo.htm
14 of 1024 http://www

### Crack open the HTML and pull some game info

This is a huge pain in the ass.  What we scraped above doesn't have all of the data.  A bunch of the content is loaded dynamically. So, we will have to simulate a browser and introduce a long sleep to let it load dynamically.  Of course, most of the time the load doesn't completely although the content we care about is there.  Ugh.

They do load reasonably if you read the html locally.  So:

1) Grab each of the partial pages from mongo and save them as a local file

2) Load that file in a browser, let it pull in the dynamic content, and then save that complete content in mongo.

3) Wait a long-ass time (~3 hours without threading).

4) Pull the complete pages and scrape the data we need from it.

*This would probably be best with Ghost driver and loadImages=False, but I'm only doing this process once and can afford to let it run overnight.*

In [52]:
def get_complete_page(page_part):
    
    # prepare to transfer the key,value pairs to the new table
    season = page_part['season']
    week = page_part['week']
    gameid = page_part['gameid']
    url = page_part['url']
    html = page_part['html']
    
    # if we've already loaded this page, skip it
    if boxscore_pages.find_one({'url':url}) > 0:
        return
    
    # save the html to a file
    with open('boxscore.html', 'w') as f:
        f.write(html)
        f.close()
    
    # load that file in a browser to finish the dynamic loads.  
    driver = webdriver.Chrome('/usr/local/share/chromedriver/chromedriver')
    driver.set_page_load_timeout(15)
    try:
        driver.get('file:///Users/zgreyn/Google%20Drive/galvanize/projects/nfl/notebooks/boxscore.html')
    except TimeoutException as ex:
        # bail out for now and come back later
        return
    
    # give it some time to load
    time.sleep(5)
    content = driver.page_source.encode('utf-8').strip()

    # store the html in MongoDB
    boxscore_pages.insert_one({'season':season, 'week':week, 'gameid':gameid, 'url': url, 'html':content})

    # quit the driver
    driver.quit()
    
    return

In [55]:
total = boxscore_pages_partial.count()
for i,page_part in enumerate(boxscore_pages_partial.find()):
    
    print i+1, 'of', total
    get_complete_page(page_part)


1 of 1024
2 of 1024
3 of 1024
4 of 1024
5 of 1024
6 of 1024
7 of 1024
8 of 1024
9 of 1024
10 of 1024
11 of 1024
12 of 1024
13 of 1024
14 of 1024
15 of 1024
16 of 1024
17 of 1024
18 of 1024
19 of 1024
20 of 1024
21 of 1024
22 of 1024
23 of 1024
24 of 1024
25 of 1024
26 of 1024
27 of 1024
28 of 1024
29 of 1024
30 of 1024
31 of 1024
32 of 1024
33 of 1024
34 of 1024
35 of 1024
36 of 1024
37 of 1024
38 of 1024
39 of 1024
40 of 1024
41 of 1024
42 of 1024
43 of 1024
44 of 1024
45 of 1024
46 of 1024
47 of 1024
48 of 1024
49 of 1024
50 of 1024
51 of 1024
52 of 1024
53 of 1024
54 of 1024
55 of 1024
56 of 1024
57 of 1024
58 of 1024
59 of 1024
60 of 1024
61 of 1024
62 of 1024
63 of 1024
64 of 1024
65 of 1024
66 of 1024
67 of 1024
68 of 1024
69 of 1024
70 of 1024
71 of 1024
72 of 1024
73 of 1024
74 of 1024
75 of 1024
76 of 1024
77 of 1024
78 of 1024
79 of 1024
80 of 1024
81 of 1024
82 of 1024
83 of 1024
84 of 1024
85 of 1024
86 of 1024
87 of 1024
88 of 1024
89 of 1024
90 of 1024
91 of 1024
92 of 10

In [149]:
def parse_play_by_play(season, week, gameid, html):

    # create the soup
    soup = BeautifulSoup(html, 'html.parser')

    # find the game info table
    game_info_table = soup.find(id='game_info')
    
    # if this game hasn't been played yet, bail out
    if game_info_table is None:
        return

    # create a dictionary of game info
    game_info = {}
    game_info['season'] = season
    game_info['week'] = week
    game_info['game_id'] = gameid
    
    # get the other game-level info
    rows = game_info_table.findAll('tr')
    for tr in rows:
        key = tr.find('th')
        value = tr.find('td')

        # ignore the header row (doesn't have a key)
        if key is not None and value is not None:
            game_info[str(key.text)] = str(value.text)

    # find the play-by-play table
    pbp_table = soup.find(id='div_pbp')

    # get the home and away teams
    for x in pbp_table.find('thead').findAll('th'):
        stat = x.get('data-stat')
        if stat == 'pbp_score_aw':
            game_info['away_team'] = x.text
        if stat == 'pbp_score_hm':
            game_info['home_team'] = x.text

    # get the plays
    plays = []
    for x in pbp_table.find('tbody').findAll('tr'):

        # copy all of the game info in
        play = copy.deepcopy(game_info)

        # put all of the play info in
        cols = x.findAll(['th','td'])
        for col in cols:
            play[col.get('data-stat')] = col.text

        # skip header rows, which only have one cell
        if len(cols) > 1:
            plays.append(play)
            
    return plays

In [150]:
# Grab the html from mongo and parse the stuff we need, then pack it into a mega dataframe
i = 0
total = boxscore_pages.count()
pbp_dfs = []
for page in boxscore_pages.find():
    
    # show progress
    i += 1
    print i, 'of', total
    
    # scrape the plays as a list of dictionaries, with game-level info included
    if 'html' in page.keys():
        plays = parse_play_by_play(page['season'], page['week'], page['gameid'], page['html'])
    
    # make sure we got some data
    if plays is not None:
    
        # create a dataframe from the plays
        plays_df = pd.DataFrame.from_dict(plays)

        # put it on the list of dataframe
        pbp_dfs.append(plays_df)
    
# combine all the play-by-play data into one
pbp_pfr = pd.concat(pbp_dfs)
print pbp_pfr.shape

1 of 1024
2 of 1024
3 of 1024
4 of 1024
5 of 1024
6 of 1024
7 of 1024
8 of 1024
9 of 1024
10 of 1024
11 of 1024
12 of 1024
13 of 1024
14 of 1024
15 of 1024
16 of 1024
17 of 1024
18 of 1024
19 of 1024
20 of 1024
21 of 1024
22 of 1024
23 of 1024
24 of 1024
25 of 1024
26 of 1024
27 of 1024
28 of 1024
29 of 1024
30 of 1024
31 of 1024
32 of 1024
33 of 1024
34 of 1024
35 of 1024
36 of 1024
37 of 1024
38 of 1024
39 of 1024
40 of 1024
41 of 1024
42 of 1024
43 of 1024
44 of 1024
45 of 1024
46 of 1024
47 of 1024
48 of 1024
49 of 1024
50 of 1024
51 of 1024
52 of 1024
53 of 1024
54 of 1024
55 of 1024
56 of 1024
57 of 1024
58 of 1024
59 of 1024
60 of 1024
61 of 1024
62 of 1024
63 of 1024
64 of 1024
65 of 1024
66 of 1024
67 of 1024
68 of 1024
69 of 1024
70 of 1024
71 of 1024
72 of 1024
73 of 1024
74 of 1024
75 of 1024
76 of 1024
77 of 1024
78 of 1024
79 of 1024
80 of 1024
81 of 1024
82 of 1024
83 of 1024
84 of 1024
85 of 1024
86 of 1024
87 of 1024
88 of 1024
89 of 1024
90 of 1024
91 of 1024
92 of 10

KeyError: 'html'

In [309]:
#xxx temp
pbp_pfr = pd.read_csv('../data/pbp-pfr.csv')

In [308]:
# clean the data a bit

# drop columns we won't use
pbp_pfr.drop(
    [
        'Over/Under',
        'Vegas Line',
        'Won Toss',
        'exp_pts_after',
        'exp_pts_before',
        'home_wp' 
    ], 
      axis=1, inplace=True,  errors='ignore')

# drop all records not in a real quarter and map OT to 5
quarters = ['1', '2', '3', '4', 'OT']
pbp_pfr = pbp_pfr[(pbp_pfr['quarter'].isin(quarters))]
pbp_pfr['quarter'] = pbp_pfr['quarter'].map({'1':'1', '2':'2','3':'3','4':'4','OT':'5'})

# drop any plays without a time and split the time remaining in to minutes and seconds
pbp_pfr = pbp_pfr[(pbp_pfr['qtr_time_remain'].notnull())]
pbp_pfr['minute'] = map(lambda x: str(x).split(':')[0], pbp_pfr['qtr_time_remain'])
pbp_pfr['second'] = map(lambda x: str(x).split(':')[1], pbp_pfr['qtr_time_remain'])
pbp_pfr.drop(['qtr_time_remain'], axis=1, inplace=True,  errors='ignore')

# the 50 yardline doesn't have a team, so give it one to smooth out parsing
pbp_pfr = pbp_pfr[(pbp_pfr['location'].notnull())]
pbp_pfr['location'].replace(' 50', 'THE 50', inplace=True)
pbp_pfr['yardlinefixed'] = map(lambda x: str(x).split()[1], pbp_pfr['location'])

# map the roof to under_roof boolean and drop the original column
roof_to_boolean = {
        'outdoors': 0,
        'dome' : 1,
        'retractable roof (open)' : 0,
        'retractable roof (closed)' : 1
    }
pbp_pfr['under_roof'] = pbp_pfr['Roof'].map(roof_to_boolean)
pbp_pfr.drop(['Roof'], axis=1, inplace=True,  errors='ignore')

# map the surface to grass/turf
surface_map = {
        'grass ' : 'grass', 
        'fieldturf ' : 'turf', 
        'fieldturf' : 'turf', 
        'sportturf' : 'turf', 
        'astroplay' : 'turf',
        'a_turf' : 'turf', 
        'matrixturf' : 'turf', 
        'grass' : 'grass'
    }
pbp_pfr['isturf'] = pbp_pfr['Surface'].map(surface_to_isturf)
pbp_pfr.drop(['Surface'], axis=1, inplace=True,  errors='ignore')

# There are 238 games (~26%) without weather data; we should replace them with 'UNKNOWN'
pbp_pfr['Weather'] = pbp_pfr['Weather'].fillna('UNKNOWN')

# some details are just missing, others are timeouts and other such plays; replace with 'UNKNOWN'.
pbp_pfr['detail'] = pbp_pfr['detail'].fillna('UNKNOWN')

# almost all of the plays missing 'down' are kick-offs, 2-pt conversions, and penaltys; drop them
pbp_pfr = pbp_pfr[(pbp_pfr['down'].notnull())]

# the two data sources use different naming conventions, so adapt to one
team_name_map =  {
        'BAL' : 'BAL', 
        'CIN' : 'CIN',
        'MIN' : 'MIN',
        'MIA' : 'MIA',
        'TAM' : 'TB',
        'ATL' : 'ATL',
        'KAN' : 'KC',
        'SEA' : 'SEA',
        'GNB' : 'GB',
        'HOU' : 'HOU',
        'NYJ' : 'NYJ',
        'SDG' : 'SD',
        'STL' : 'LA',
        'JAX' : 'JAX',
        'SFO' : 'SF',
        'PIT' : 'PIT',
        'ARI' : 'ARI',
        'NYG' : 'NYG',
        'DET' : 'DET',
        'CLE' : 'CLE',
        'BUF' : 'BUF',
        'IND' : 'IND',
        'CHI' : 'CHI',
        'OAK' : 'OAK',
        'DAL' : 'DAL',
        'PHI' : 'PHI',
        'WAS' : 'WAS',
        'NOR' : 'NO',
        'NWE' : 'NE',
        'CAR' : 'CAR',
        'DEN' : 'DEN',
        'TEN' : 'TEN',
        'LAR' : 'LA'
    }
pbp_pfr['away_team'] = pbp_pfr['away_team'].map(team_name_map)
pbp_pfr['home_team'] = pbp_pfr['home_team'].map(team_name_map)

# change the types of a few columns
pbp_pfr['down'] = pbp_pfr['down'].astype("int64")
pbp_pfr['pbp_score_aw'] = pbp_pfr['pbp_score_aw'].astype("int64")
pbp_pfr['pbp_score_hm'] = pbp_pfr['pbp_score_hm'].astype("int64")
pbp_pfr['quarter'] = pbp_pfr['quarter'].astype("int64")
pbp_pfr['yds_to_go'] = pbp_pfr['yds_to_go'].astype("int64")
pbp_pfr['minute'] = pbp_pfr['minute'].astype("int64")
pbp_pfr['second'] = pbp_pfr['second'].astype("int64")
pbp_pfr['yardlinefixed'] = pbp_pfr['yardlinefixed'].astype("int64")

print pbp_pfr.shape

KeyError: 'qtr_time_remain'

In [353]:
# output the compiled data frame to save it
pbp_pfr.to_csv('../data/pbp-pfr.csv', index=False)

### Next, go through each record in our main data frame and try to find the matching record in the new one.  Append the current score, home team, away team, weather, roof, and surface to each record.

In [404]:
# add some empty columns to fill
pbp['ISTURF'] = np.nan
pbp['UNDERROOF'] = np.nan
pbp['WEATHER'] = np.nan
pbp['HOMETEAM'] = np.nan
pbp['AWAYTEAM'] = np.nan
pbp['HOMESCORE'] = np.nan
pbp['AWAYSCORE'] = np.nan

In [ ]:
# for each play in our main dataframe, find the corresponding play in the other
for row in pbp.iterrows():
    play = row[1]
    matching_play = pbp_pfr[
            (pbp_pfr['season'] == play['SEASONYEAR']) & 
            (pbp_pfr['quarter'] == play['QUARTER']) & 
            (pbp_pfr['minute'] == play['MINUTE']) & 
            (pbp_pfr['second'] == play['SECOND']) & 
            (pbp_pfr['down'] == play['DOWN']) & 
            (pbp_pfr['yds_to_go'] == play['TOGO']) &
            (pbp_pfr['yardlinefixed'] == play['YARDLINEFIXED']) &
            ~(pbp_pfr['detail'].str.startswith('Penalty')) &
            (((pbp_pfr['home_team'] == play['OFFENSETEAM']) & (pbp_pfr['away_team'] == play['DEFENSETEAM'])) | ((pbp_pfr['home_team'] == play['DEFENSETEAM']) & (pbp_pfr['away_team'] == play['OFFENSETEAM'])))
        ]
    
    if len(matching_play.index) == 1:
        matching_play = matching_play.iloc[0]
        play['ISTURF'] = matching_play['isturf']
        play['UNDERROOF'] = matching_play['under_roof']
        play['WEATHER'] = matching_play['Weather']
        play['HOMETEAM'] = matching_play['home_team']
        play['AWAYTEAM'] = matching_play['away_team']
        play['HOMESCORE'] = matching_play['pbp_score_hm']
        play['AWAYSCORE'] = matching_play['pbp_score_aw']

In [390]:
#xxx temp
pbp.to_csv('../data/temp-pbp-pfr-combined.csv', index=False)

In [ ]:
#xxx temp
pbp_pfr = pd.read_csv('../data/temp-pbp-pfr-combined.csv')

#### There will be some records with NaN's in them.  check it out because they probably had 0 matches in the other dataframe.   The one i saw was a penalty no-play, so there might be others we can discard.

In [ ]:
pbp.shape()

In [ ]:
# Import smtplib for the actual sending function
import smtplib

# Import the email modules we'll need
from email.mime.text import MIMEText

# Create a text/plain message
msg = MIMEText("Done!")
msg['Subject'] = 'Combine operation is done'
msg['From'] = 'zane.reynolds@gmail.com'
msg['To'] = 'zane.reynolds@gmail.com'

# Send the message via our own SMTP server, but don't include the
# envelope header.
s = smtplib.SMTP('localhost')
s.sendmail(me, [you], msg.as_string())
s.quit()

### Finally, modify the clean and prep methods to include the new features (score should be the differential/margin relative to the offense).

1) Convert HOMETEAM / AWAYTEAM to ISATHOME relative to offense

2) Convert HOMESCORE / AWAYSCORE to scoring margin relative to offense

3) Break temperature, humidity, and windspeed out where we can.  There's no precipitation info.

In [ ]:
pbp['ISATHOME'] = pbp.apply(lambda x: 1 if x.HOMETEAM == x.OFFENSETEAM else 0, axis=1)

In [ ]:
def compute_margin(play):
    if play['ISATHOME'] == 1:
        return play['pbp_score_hm'] - play['pbp_score_aw']
    else:
        return play['pbp_score_aw'] - play['pbp_score_hm']

In [ ]:
pbp['SCORINGMARGIN'] = pbp.apply(lambda x: compute_margin(x), axis=1)

In [ ]:
def parse_weather(wstring):
    '''
    INPUT: A game weather string scraped from pro-football-reference.com
    OUTPUT: dict{'TEMPERATURE':(F degrees), 'HUMIDITY': (%), 'WINDSPEED': (mph)}
    
    Example inputs:
    '36 degrees relative humidity 91%, wind 16 mph, wind chill 26',
    '37 degrees relative humidity 39%, wind 2 mph',
    '37 degrees relative humidity 56%, no wind',
    '''
    
    ret_values = {}
    ret_values['TEMPERATURE'] = 'UNKOWN'
    ret_values['HUMIDITY'] = 'UNKNOWN'
    ret_values['WINDSPEED'] = 'UNKNOWN'
    
    # if the value was missing, propagate that back
    if wstring == 'UNKNOWN':
        return ret_values
    
    # add a comma between temp and humidity to make the parts cleaner
    wstring = wstring.replace('degrees relative', 'degrees, relative')
    
    # split into parts
    parts = wstring.split(',')
    
    # part 0 is the temperature
    ret_values['TEMPERATURE'] = parts[0].split()[0]
    
    # part 1 is the humidity
    ret_values['HUMIDITY'] = parts[1][:-1].split()[-1]
    
    # part 3 is the windspeed
    if parts[2] == 'no wind':
        ret_values['WINDSPEED'] = '0'
    else:
        ret_values['WINDSPEED'] = parts[2].split()[1]
    
    # sometimes, there's a part 4, which is wind chill, but we will ignore that for now
    
    return ret_values

In [ ]:
pbp = pd.concat([pbp, pbp.Weather.apply(lambda x: pd.Series(parse_weather(x)))], axis=1)